# Parser Teks Dokumen dan Pengekstrak Kata Kunci Keuangan (Tahunan)

Notebook ini bertujuan untuk mem-parsing teks dari berbagai jenis dokumen (gambar, TXT, DOCX, PDF) dan kemudian mengekstrak istilah-istilah keuangan spesifik beserta nilainya, dengan fokus pada **data tahun pelaporan terbaru**. Proses ini melibatkan beberapa teknologi dan fitur utama:
- **OCR (Optical Character Recognition)**: Untuk mengekstrak teks dari gambar dan PDF berbasis gambar.
- **Pra-pemrosesan Gambar**: Sebelum OCR, gambar diproses melalui beberapa tahap (konversi ke skala abu, penghilangan derau, binerisasi, dan percobaan pelurusan kemiringan) untuk meningkatkan kualitas OCR.
- **Pemrosesan Paralel untuk PDF**: Halaman PDF yang memerlukan OCR diproses secara paralel untuk mempercepat ekstraksi.
- **Mekanisme Caching**: Hasil parsing PDF disimpan dalam cache untuk menghindari pemrosesan ulang file yang sama jika tidak ada perubahan.
- **Ekstraksi Kata Kunci Bertarget**: Mencari istilah keuangan yang telah ditentukan (dalam Bahasa Indonesia) dan mencoba mengidentifikasi nilai numerik yang berasosiasi dengan tahun pelaporan terbaru yang terdeteksi dalam dokumen.
- **Normalisasi Nilai**: Nilai keuangan yang diekstrak dinormalisasi ke format float.
- **Output JSON**: Hasil akhir ekstraksi kata kunci dan nilainya disajikan dalam format JSON.

Pastikan semua skrip Python pendukung (`parser_gambar.py`, `parser_dokumen_teks.py`, `parser_pdf.py`, `pengekstrak_kata_kunci.py`, `utilitas_cache.py`) berada di direktori yang sama dengan notebook ini atau terinstal dalam lingkungan Python Anda.


In [1]:
# Langkah Pengaturan Awal

# 1. Impor Pustaka dan Modul Kustom
# Pastikan semua skrip Python (.py) yang disebutkan di bawah ini
# berada di direktori yang sama dengan notebook ini.

import os
import json
import nltk

# Impor fungsi-fungsi dari modul-modul utilitas kita
try:
    from SaranaModule.parser_gambar import ekstrak_teks_dari_gambar
    from SaranaModule.parser_dokumen_teks import ekstrak_teks_dari_txt, ekstrak_teks_dari_docx
    from SaranaModule.parser_pdf import ekstrak_teks_dari_pdf # Fungsi ini menggunakan ekstrak_teks_dari_gambar untuk OCR
    from SaranaModule.parser_tabular import ekstrak_data_dari_xlsx, ekstrak_data_dari_csv
    from SaranaModule.pengekstrak_kata_kunci import (
        DAFTAR_KATA_KUNCI_KEUANGAN_DEFAULT, # Daftar kata kunci default
        identifikasi_tahun_pelaporan,       # Untuk menemukan tahun dalam dokumen
        ekstrak_data_keuangan_tahunan,      # Fungsi ekstraksi utama yang baru
        format_ke_json,                     # Untuk output JSON
        normalisasi_nilai_keuangan,          # Untuk membersihkan nilai angka (jika ingin diuji terpisah)
        deteksi_pengali_global              # Untuk mendeteksi pengali global seperti 'juta', 'miliar', dll.
    )
    print("Modul-modul kustom berhasil diimpor.")
except ImportError as e:
    print(f"Error mengimpor modul kustom: {e}")
    print("Pastikan semua file .py (parser_gambar, parser_dokumen_teks, parser_pdf, pengekstrak_kata_kunci, utilitas_cache) berada di direktori yang sama.")


try:
    nltk.data.find('corpora/wordnet.zip')
    print("Resource NLTK (wordnet) sudah ada.")
except LookupError:
    import ssl
    ssl._create_default_https_context = ssl._create_unverified_context
    print("Resource NLTK (wordnet) tidak ditemukan, mengunduh...")
    nltk.download('wordnet')
    nltk.download('punkt_tab')
    nltk.download('omw-1.4') # wordnet multilingual
    nltk.download('punkt')   # untuk tokenisasi
    nltk.download('stopwords') # untuk stopwords
    print("Resource NLTK (wordnet) sudah terunduh.")

except Exception as e:
     print(f"Error terkait NLTK: {e}")

print("\nPengaturan Selesai. Anda dapat melanjutkan ke sel Konfigurasi.")

Berhasil memuat stopwords Bahasa Indonesia.
WordNetLemmatizer berhasil diinisialisasi.
Tokenizer 'punkt' tampaknya tersedia.
Modul-modul kustom berhasil diimpor.
Resource NLTK (wordnet) sudah ada.

Pengaturan Selesai. Anda dapat melanjutkan ke sel Konfigurasi.


## Penjelasan Fitur Utama

Sebelum melanjutkan ke konfigurasi, berikut adalah ringkasan singkat tentang beberapa fitur utama yang digunakan dalam notebook ini:

*   **Pra-pemrosesan Gambar untuk OCR**: Jika dokumen Anda adalah gambar atau PDF yang berisi halaman gambar, kualitas OCR sangat penting. Modul `parser_gambar.py` kini menyertakan langkah-langkah seperti konversi ke skala abu, penghilangan derau (noise), dan binerisasi (mengubah gambar menjadi hitam-putih) untuk meningkatkan akurasi Tesseract OCR. Implementasi dasar untuk pelurusan kemiringan (deskewing) juga ada, meskipun mungkin memerlukan penyesuaian lebih lanjut untuk kasus yang kompleks.
*   **Pemrosesan Paralel untuk PDF**: Untuk mempercepat ekstraksi teks dari PDF yang memiliki banyak halaman berbasis gambar (yang memerlukan OCR), `parser_pdf.py` menggunakan `ThreadPoolExecutor`. Ini memungkinkan beberapa halaman diproses secara bersamaan, mengurangi waktu tunggu total.
*   **Caching Hasil Parsing**: Untuk menghindari pemrosesan ulang file PDF yang sama berulang kali (yang bisa memakan waktu), `parser_pdf.py` kini terintegrasi dengan mekanisme caching (`utilitas_cache.py`). Hasil ekstraksi teks dari sebuah PDF akan disimpan dalam cache (default di direktori `.cache_parser_dokumen`). Jika Anda memproses PDF yang sama lagi dan file tersebut tidak berubah (berdasarkan path dan timestamp modifikasi terakhir), hasilnya akan diambil dari cache, yang jauh lebih cepat. Anda bisa membersihkan cache ini secara manual atau menggunakan fungsi `bersihkan_cache_lama` (jika ingin diimplementasikan lebih lanjut).
*   **Logika Ekstraksi Nilai Berbasis Tahun**: Fungsi `ekstrak_data_keuangan_tahunan` dalam `pengekstrak_kata_kunci.py` dirancang untuk pertama-tama mengidentifikasi tahun pelaporan utama dalam dokumen. Kemudian, saat mencari nilai untuk kata kunci keuangan, ia akan mencoba memprioritaskan angka yang berasosiasi dengan tahun pelaporan tersebut dan membedakannya dari angka untuk tahun sebelumnya, jika keduanya muncul berdekatan.


In [2]:
# --- Konfigurasi Pengguna ---

# 1. Tentukan path ke DIREKTORI yang berisi dokumen yang ingin Anda proses.
# Contoh: "train_documents/" untuk data periode t, atau "train_documents_t_minus_1/" untuk data periode t-1.
path_direktori_dokumen_input = "train_documents/"

# 2. Tentukan nama file JSON output untuk menyimpan hasil ekstraksi.
# Contoh: "hasil_ekstraksi_semua_dokumen.json" untuk periode t,
#         "hasil_ekstraksi_semua_dokumen_t_minus_1.json" untuk periode t-1.
nama_file_json_output = "hasil_ekstraksi_semua_dokumen.json"
# Direktori output akan tetap "OutputSarana/" secara default, jadi path lengkapnya akan menjadi "OutputSarana/nama_file_json_output".

# 3. Definisikan atau Modifikasi Kata Kunci yang Akan Diekstrak
# `konfigurasi_kata_kunci_target` adalah list kamus (dictionary).
# Setiap kamus harus memiliki:
#    - 'kata_dasar': Nama kanonis untuk kata kunci tersebut (misalnya, "Laba Bersih"). Ini akan menjadi kunci dalam output JSON.
#    - 'variasi': List berisi berbagai cara penulisan atau sinonim kata kunci tersebut yang mungkin muncul di dokumen.
#
# Anda bisa menggunakan daftar default yang diimpor (`DAFTAR_KATA_KUNCI_KEUANGAN_DEFAULT`)
# atau membuat/memodifikasi daftar Anda sendiri di bawah ini.
#
# Untuk menggunakan daftar default:
from SaranaModule.pengekstrak_kata_kunci import DAFTAR_KATA_KUNCI_KEUANGAN_DEFAULT
konfigurasi_kata_kunci_target = DAFTAR_KATA_KUNCI_KEUANGAN_DEFAULT

# 3. (Opsional) Konfigurasi Direktori Cache untuk PDF Parser
# Jika Anda ingin parser PDF menggunakan direktori cache selain default (`.cache_parser_dokumen`),
# Anda bisa menentukan path-nya di sini. Jika tidak, biarkan `None` untuk menggunakan default.
direktori_cache_pdf_kustom = ".cache_parsing_dokumen"

# --- Akhir Konfigurasi Pengguna ---

# Validasi awal konfigurasi
if 'path_direktori_dokumen_input' not in locals() or not path_direktori_dokumen_input:
    print("PERINGATAN: 'path_direktori_dokumen_input' belum diatur atau kosong.")
    print("Mohon perbarui variabel 'path_direktori_dokumen_input' di atas dengan path ke DIREKTORI yang berisi dokumen-dokumen yang ingin Anda proses.")
elif not os.path.isdir(path_direktori_dokumen_input):
    print(f"ERROR: Path direktori input yang ditentukan ('{path_direktori_dokumen_input}') bukan direktori atau tidak ditemukan.")
    print("Mohon periksa kembali 'path_direktori_dokumen_input' dan pastikan itu adalah direktori yang valid dan ada.")
elif 'nama_file_json_output' not in locals() or not nama_file_json_output:
    print("PERINGATAN: 'nama_file_json_output' belum diatur atau kosong.")
    print("Mohon perbarui variabel 'nama_file_json_output' untuk menentukan nama file hasil ekstraksi.")
else:
    print(f"Konfigurasi dimuat. Direktori dokumen yang akan diproses: {path_direktori_dokumen_input}")
    print(f"Hasil ekstraksi akan disimpan ke: OutputSarana/{nama_file_json_output}")
    print(f"Kata kunci yang akan dicari: {[item['kata_dasar'] for item in konfigurasi_kata_kunci_target]}")
    if direktori_cache_pdf_kustom:
        print(f"Direktori cache PDF kustom diatur ke: {direktori_cache_pdf_kustom}")

# Inisialisasi list untuk menyimpan semua hasil ekstraksi dari semua dokumen
semua_hasil_ekstraksi = []


Konfigurasi dimuat. Direktori dokumen yang akan diproses: train_documents/
Hasil ekstraksi akan disimpan ke: OutputSarana/hasil_ekstraksi_semua_dokumen.json
Kata kunci yang akan dicari: ['Jumlah aset lancar', 'Jumlah aset tidak lancar', 'Jumlah liabilitas jangka pendek', 'Jumlah liabilitas jangka panjang', 'Jumlah ekuitas', 'Jumlah liabilitas dan ekuitas', 'Pendapatan bersih', 'Beban pokok pendapatan', 'Laba bruto', 'Laba sebelum pajak penghasilan', 'Beban pajak penghasilan', 'Laba tahun berjalan', 'Jumlah aset', 'Piutang usaha', 'Aset tetap', 'Aset tetap bruto', 'Akumulasi penyusutan', 'Modal kerja bersih', 'Jumlah liabilitas', 'Laba ditahan', 'Beban bunga', 'Beban penyusutan', 'Beban penjualan', 'Beban administrasi dan umum', 'Beban usaha', 'Arus kas bersih yang diperoleh dari aktivitas operasi', 'Arus kas bersih yang diperoleh dari aktivitas investasi', 'Arus kas bersih yang digunakan untuk aktivitas pendanaan', 'Piutang usaha tahun lalu', 'Pendapatan bersih tahun lalu', 'Laba kotor

In [3]:
# Langkah ini akan melakukan iterasi melalui semua dokumen yang didukung dalam direktori yang ditentukan,
# mengekstrak teks dari masing-masing dokumen, dan kemudian mengekstrak kata kunci.

# Pastikan path_direktori_dokumen_input (direktori) telah dikonfigurasi dengan benar dan merupakan direktori
if 'path_direktori_dokumen_input' not in locals() or not path_direktori_dokumen_input or not os.path.isdir(path_direktori_dokumen_input) or \
    'nama_file_json_output' not in locals() or not nama_file_json_output:
    pesan_error_global = "Error: Konfigurasi 'path_direktori_dokumen_input' atau 'nama_file_json_output' tidak valid. Silakan perbarui di sel Konfigurasi."
    if 'path_direktori_dokumen_input' in locals() and (not path_direktori_dokumen_input):
        pesan_error_global = "Error: 'path_direktori_dokumen_input' (direktori) belum diatur atau kosong. Silakan perbarui di sel Konfigurasi."
    elif 'path_direktori_dokumen_input' in locals() and not os.path.isdir(path_direktori_dokumen_input):
        pesan_error_global = f"Error: Path '{path_direktori_dokumen_input}' bukan direktori yang valid atau tidak ditemukan. Mohon verifikasi path di sel Konfigurasi."
    elif 'nama_file_json_output' not in locals() or not nama_file_json_output:
        pesan_error_global = "Error: 'nama_file_json_output' belum diatur atau kosong. Silakan perbarui di sel Konfigurasi."
    
    print(pesan_error_global)
    # MODIFIKASI: Inisialisasi list t dan t-1 jika ada error konfigurasi global
    if 'semua_hasil_ekstraksi_t' not in locals(): semua_hasil_ekstraksi_t = [] 
    if 'semua_hasil_ekstraksi_t_minus_1' not in locals(): semua_hasil_ekstraksi_t_minus_1 = [] 
    semua_hasil_ekstraksi_t.append({
        "nama_file": "KONFIGURASI_ERROR",
        "hasil_ekstraksi": {"error_global_konfigurasi": pesan_error_global}
    })
else:
    supported_extensions = ['.pdf', '.jpg', '.jpeg', '.png', '.tiff', '.bmp', '.gif', '.txt', '.docx', '.xlsx', '.csv']
    print(f"Mencari dokumen dengan ekstensi yang didukung: {', '.join(supported_extensions)} di direktori: {path_direktori_dokumen_input}")
    
    # MODIFIKASI: Inisialisasi dua list untuk hasil t dan t-1
    semua_hasil_ekstraksi_t = []
    semua_hasil_ekstraksi_t_minus_1 = []

    files_to_process = []
    try:
        files_to_process = [
            f for f in os.listdir(path_direktori_dokumen_input) 
            if os.path.isfile(os.path.join(path_direktori_dokumen_input, f)) and 
            os.path.splitext(f)[1].lower() in supported_extensions
        ]
    except FileNotFoundError:
        print(f"Error: Direktori '{path_direktori_dokumen_input}' tidak ditemukan. Periksa konfigurasi path_direktori_dokumen_input.")
        semua_hasil_ekstraksi_t.append({"nama_file": "DIREKTORI_TIDAK_DITEMUKAN", "hasil_ekstraksi": {"error_direktori": f"Direktori '{path_direktori_dokumen_input}' tidak ditemukan."}})
    except Exception as e_listdir:
        print(f"Error saat mengakses direktori '{path_direktori_dokumen_input}': {e_listdir}")
        semua_hasil_ekstraksi_t.append({"nama_file": "AKSES_DIREKTORI_ERROR", "hasil_ekstraksi": {"error_direktori": f"Error mengakses '{path_direktori_dokumen_input}': {e_listdir}"}})

    if not files_to_process:
        msg_no_files = f"Tidak ada dokumen dengan ekstensi yang didukung ditemukan di {path_direktori_dokumen_input}"
        print(msg_no_files)
        # Pastikan pesan 'tidak ada file' hanya ditambahkan jika belum ada error konfigurasi/direktori sebelumnya
        if not any(d.get("nama_file") in ["KONFIGURASI_ERROR", "DIREKTORI_TIDAK_DITEMUKAN", "AKSES_DIREKTORI_ERROR"] for d in semua_hasil_ekstraksi_t):
            semua_hasil_ekstraksi_t.append({"nama_file": "TIDAK_ADA_FILE", "hasil_ekstraksi": {"info": msg_no_files}})
    else:
        print(f"Ditemukan {len(files_to_process)} dokumen untuk diproses: {files_to_process}")

        for nama_file_dokumen in files_to_process:
            current_file_path = os.path.join(path_direktori_dokumen_input, nama_file_dokumen)
            print(f"\n--- Memulai pemrosesan untuk dokumen: {nama_file_dokumen} ---")
            
            teks_hasil_ekstraksi_file = "" # Inisialisasi untuk setiap file
            kamus_t_only = {} # MODIFIKASI: Untuk hasil t
            kamus_t_minus_1_only = {} # MODIFIKASI: Untuk hasil t-1
            
            ekstensi_file = os.path.splitext(nama_file_dokumen)[1].lower()
            print(f"Tipe berkas terdeteksi: {ekstensi_file} untuk {nama_file_dokumen}")

            try:
                if ekstensi_file == '.pdf':
                    dir_cache = direktori_cache_pdf_kustom if 'direktori_cache_pdf_kustom' in locals() and direktori_cache_pdf_kustom else None
                    teks_hasil_ekstraksi_file = ekstrak_teks_dari_pdf(current_file_path, ekstrak_teks_dari_gambar, direktori_cache_kustom=dir_cache)
                elif ekstensi_file in ['.jpg', '.jpeg', '.png', '.tiff', '.bmp', '.gif']:
                    teks_hasil_ekstraksi_file = ekstrak_teks_dari_gambar(current_file_path)
                elif ekstensi_file == '.xlsx':
                    teks_hasil_ekstraksi_file = ekstrak_data_dari_xlsx(current_file_path)
                elif ekstensi_file == '.csv':
                    teks_hasil_ekstraksi_file = ekstrak_data_dari_csv(current_file_path)
                elif ekstensi_file == '.txt':
                    teks_hasil_ekstraksi_file = ekstrak_teks_dari_txt(current_file_path)
                elif ekstensi_file == '.docx':
                    teks_hasil_ekstraksi_file = ekstrak_teks_dari_docx(current_file_path)
            except Exception as e_parse:
                error_msg_parse = f"Error selama parsing dokumen '{nama_file_dokumen}': {str(e_parse)}"
                print(error_msg_parse)
                teks_hasil_ekstraksi_file = error_msg_parse 

            if not teks_hasil_ekstraksi_file.startswith("Error:") and teks_hasil_ekstraksi_file.strip():
                print(f"Parsing dokumen '{nama_file_dokumen}' selesai. Total karakter: {len(teks_hasil_ekstraksi_file)}")
                print(f"Memulai ekstraksi kata kunci untuk: {nama_file_dokumen}")
                pengali_dokumen_file = 1.0 
                print_output_pengali_file = []
                try:
                    pengali_dokumen_file = deteksi_pengali_global(teks_hasil_ekstraksi_file)
                    print_output_pengali_file.append(f"Pengali terdeteksi untuk '{nama_file_dokumen}': {pengali_dokumen_file}")
                except Exception as e_pengali:
                    print_output_pengali_file.append(f"Error saat deteksi pengali untuk '{nama_file_dokumen}': {str(e_pengali)}. Menggunakan default 1.0.")
                    pengali_dokumen_file = 1.0
                for msg in print_output_pengali_file: print(msg)
                if 'konfigurasi_kata_kunci_target' not in locals() or not konfigurasi_kata_kunci_target:
                    pesan_error_konfig_loop = "Error krusial: 'konfigurasi_kata_kunci_target' tidak terdefinisi. Periksa sel Konfigurasi."
                    print(pesan_error_konfig_loop)
                    kamus_t_only = {"error_konfigurasi_global": pesan_error_konfig_loop} # MODIFIKASI
                else:
                    try:
                        kamus_hasil_ekstraksi_file_lengkap = ekstrak_data_keuangan_tahunan(
                            teks_hasil_ekstraksi_file, 
                            konfigurasi_kata_kunci_target, 
                            pengali_global=pengali_dokumen_file
                        )
                        print(f"Ekstraksi kata kunci untuk '{nama_file_dokumen}' selesai.")
                        # MODIFIKASI: Memecah hasil_ekstraksi_file_lengkap menjadi t_only dan t_minus_1_only
                        if isinstance(kamus_hasil_ekstraksi_file_lengkap, dict):
                            for key, values_dict in kamus_hasil_ekstraksi_file_lengkap.items():
                                if isinstance(values_dict, dict):
                                    if values_dict.get('t') is not None:
                                        kamus_t_only[key] = values_dict['t']
                                    if values_dict.get('t-1') is not None:
                                        kamus_t_minus_1_only[key] = values_dict['t-1']
                                else: # Fallback jika struktur tidak seperti yang diharapkan
                                    kamus_t_only[key] = values_dict 
                        else: # Jika hasil ekstraksi bukan dict (misal, error string dari tahap sebelumnya)
                             kamus_t_only = kamus_hasil_ekstraksi_file_lengkap
                    except Exception as e_ekstraksi:
                        pesan_error_ekstraksi_loop = f"Error selama proses ekstraksi kata kunci untuk '{nama_file_dokumen}': {str(e_ekstraksi)}"
                        print(pesan_error_ekstraksi_loop)
                        kamus_t_only = {"error_runtime_ekstraksi": pesan_error_ekstraksi_loop} # MODIFIKASI
                        kamus_t_minus_1_only = {} # Tetap kosong saat error
            elif not teks_hasil_ekstraksi_file.strip() and not teks_hasil_ekstraksi_file.startswith("Error:"):
                info_msg = f"Info: Teks yang diekstrak dari '{nama_file_dokumen}' kosong atau hanya spasi putih."
                print(info_msg)
                kamus_t_only = {"info_parsing": info_msg} # MODIFIKASI
                kamus_t_minus_1_only = {}
            else: # Ada error dari parsing (teks_hasil_ekstraksi_file sudah berisi pesan error)
                kamus_t_only = {"error_parsing": teks_hasil_ekstraksi_file} # MODIFIKASI
                kamus_t_minus_1_only = {}

            # MODIFIKASI: Simpan ke list yang sesuai
            if kamus_t_only: # Hanya tambahkan jika ada isinya (termasuk error/info)
                semua_hasil_ekstraksi_t.append({
                    "nama_file": nama_file_dokumen,
                    "hasil_ekstraksi": kamus_t_only
                })
            
            if kamus_t_minus_1_only: # Hanya tambahkan jika ada isinya
                 semua_hasil_ekstraksi_t_minus_1.append({
                    "nama_file": nama_file_dokumen,
                    "hasil_ekstraksi": kamus_t_minus_1_only
                })
            print(f"--- Pemrosesan untuk dokumen: {nama_file_dokumen} selesai. Hasil disimpan (secara terpisah untuk t dan t-1). ---")
            
        print("\n=== Semua dokumen dalam direktori telah diproses. ===")

# MODIFIKASI: Ringkasan setelah loop selesai (opsional, bisa disesuaikan untuk dua list)
if 'semua_hasil_ekstraksi_t' in locals() and semua_hasil_ekstraksi_t:
    print(f"Total item dalam 'semua_hasil_ekstraksi_t': {len(semua_hasil_ekstraksi_t)}")
    # Tambahkan loop serupa untuk semua_hasil_ekstraksi_t_minus_1 jika perlu detail status
else:
    print("Tidak ada hasil ekstraksi (t) yang tersimpan atau list kosong. Periksa log di atas.")
if 'semua_hasil_ekstraksi_t_minus_1' in locals() and semua_hasil_ekstraksi_t_minus_1:
    print(f"Total item dalam 'semua_hasil_ekstraksi_t_minus_1': {len(semua_hasil_ekstraksi_t_minus_1)}")
else:
    print("Tidak ada hasil ekstraksi (t-1) yang tersimpan atau list kosong.")


Mencari dokumen dengan ekstensi yang didukung: .pdf, .jpg, .jpeg, .png, .tiff, .bmp, .gif, .txt, .docx, .xlsx, .csv di direktori: train_documents/
Ditemukan 1 dokumen untuk diproses: ['astra_lapkeu.pdf']

--- Memulai pemrosesan untuk dokumen: astra_lapkeu.pdf ---
Tipe berkas terdeteksi: .pdf untuk astra_lapkeu.pdf
Mengambil hasil dari cache untuk: astra_lapkeu.pdf
Parsing dokumen 'astra_lapkeu.pdf' selesai. Total karakter: 12483
Memulai ekstraksi kata kunci untuk: astra_lapkeu.pdf
Pengali terdeteksi untuk 'astra_lapkeu.pdf': 1000000000.0
Ekstraksi kata kunci untuk 'astra_lapkeu.pdf' selesai.
--- Pemrosesan untuk dokumen: astra_lapkeu.pdf selesai. Hasil disimpan (secara terpisah untuk t dan t-1). ---

=== Semua dokumen dalam direktori telah diproses. ===
Total item dalam 'semua_hasil_ekstraksi_t': 1
Total item dalam 'semua_hasil_ekstraksi_t_minus_1': 1


In [4]:
# MODIFIKASI: Langkah ini sekarang akan memformat dan menyimpan dua set data: t dan t-1.

output_dir_agregat = "Output/Sarana"
os.makedirs(output_dir_agregat, exist_ok=True) # Buat direktori jika belum ada

# --- PENYIMPANAN DATA T ---
if 'nama_file_json_output' in locals() and nama_file_json_output:
    data_untuk_json_t = []
    if 'semua_hasil_ekstraksi_t' in locals() and isinstance(semua_hasil_ekstraksi_t, list):
        if semua_hasil_ekstraksi_t: # Jika list tidak kosong
            print(f"'semua_hasil_ekstraksi_t' ditemukan dengan {len(semua_hasil_ekstraksi_t)} item. Akan diformat ke JSON.")
            data_untuk_json_t = semua_hasil_ekstraksi_t
        else: # List ada tapi kosong
            print("Info: 'semua_hasil_ekstraksi_t' adalah list kosong (tidak ada file diproses atau file tidak menghasilkan output).")
            data_untuk_json_t.append({"info": "Tidak ada data (t) yang diekstrak dari dokumen manapun."})
    else: # Variabel tidak ada atau bukan list
        print("Error: 'semua_hasil_ekstraksi_t' tidak ditemukan atau tidak valid. Proses ekstraksi mungkin gagal total.")
        data_untuk_json_t.append({"error_kritis": "Variabel 'semua_hasil_ekstraksi_t' tidak tersedia atau tidak valid."})

    try:
        output_json_t = json.dumps(data_untuk_json_t, indent=4, ensure_ascii=False)
        print("\n--- Output Agregat Final (JSON untuk data t) ---")
        if len(output_json_t) > 1000: print(output_json_t[:1000] + "... (output t dipotong)")
        else: print(output_json_t)
        
        full_output_path_t = os.path.join(output_dir_agregat, nama_file_json_output) # Nama file asli untuk data t
        with open(full_output_path_t, "w", encoding="utf-8") as f:
            f.write(output_json_t)
        print(f"\nMenyimpan hasil ekstraksi (t) semua dokumen ke berkas JSON: {full_output_path_t}")
    except Exception as e_write_t:
        print(f"\nERROR: Gagal menyimpan berkas JSON (t) di '{full_output_path_t}': {e_write_t}")
else:
    print("\nPERINGATAN: 'nama_file_json_output' tidak terdefinisi. Hasil JSON (t) tidak disimpan.")

# --- PENYIMPANAN DATA T-1 ---
if 'nama_file_json_output' in locals() and nama_file_json_output:
    data_untuk_json_t_minus_1 = []
    save_t_minus_1_file = False # Flag untuk menentukan apakah file t-1 perlu disimpan

    if 'semua_hasil_ekstraksi_t_minus_1' in locals() and isinstance(semua_hasil_ekstraksi_t_minus_1, list):
        if semua_hasil_ekstraksi_t_minus_1: # Jika list tidak kosong
            print(f"'semua_hasil_ekstraksi_t_minus_1' ditemukan dengan {len(semua_hasil_ekstraksi_t_minus_1)} item. Akan diformat ke JSON.")
            data_untuk_json_t_minus_1 = semua_hasil_ekstraksi_t_minus_1
            save_t_minus_1_file = True
        else: # List ada tapi kosong
            print("Info: 'semua_hasil_ekstraksi_t_minus_1' adalah list kosong (tidak ada data t-1 yang diekstrak).")
            # Tidak membuat file jika memang tidak ada data t-1 sama sekali
    else: # Variabel tidak ada atau bukan list
        print("Error: 'semua_hasil_ekstraksi_t_minus_1' tidak ditemukan atau tidak valid.")
        # data_untuk_json_t_minus_1.append({"error_kritis": "Variabel 'semua_hasil_ekstraksi_t_minus_1' tidak tersedia."}) # Hindari menyimpan file error jika listnya tidak ada

    if save_t_minus_1_file:
        try:
            output_json_t_minus_1 = json.dumps(data_untuk_json_t_minus_1, indent=4, ensure_ascii=False)
            print("\n--- Output Agregat Final (JSON untuk data t-1) ---")
            if len(output_json_t_minus_1) > 1000: print(output_json_t_minus_1[:1000] + "... (output t-1 dipotong)")
            else: print(output_json_t_minus_1)

            base, ext = os.path.splitext(nama_file_json_output)
            nama_file_t_minus_1 = f"{base}_t_minus_1{ext}"
            full_output_path_t_minus_1 = os.path.join(output_dir_agregat, nama_file_t_minus_1)
            
            with open(full_output_path_t_minus_1, "w", encoding="utf-8") as f:
                f.write(output_json_t_minus_1)
            print(f"\nMenyimpan hasil ekstraksi (t-1) semua dokumen ke berkas JSON: {full_output_path_t_minus_1}")

        except Exception as e_write_t_minus_1:
            print(f"\nERROR: Gagal menyimpan berkas JSON (t-1) di '{full_output_path_t_minus_1}': {e_write_t_minus_1}")
    elif not ('semua_hasil_ekstraksi_t_minus_1' in locals() and semua_hasil_ekstraksi_t_minus_1):
        print("\nInfo: Tidak ada data (t-1) yang diproses atau valid untuk disimpan.")
else:
    print("\nPERINGATAN: 'nama_file_json_output' tidak terdefinisi. Hasil JSON (t-1) tidak disimpan.")


'semua_hasil_ekstraksi_t' ditemukan dengan 1 item. Akan diformat ke JSON.

--- Output Agregat Final (JSON untuk data t) ---
[
    {
        "nama_file": "astra_lapkeu.pdf",
        "hasil_ekstraksi": {
            "Jumlah aset lancar": 19238000000000.0,
            "Jumlah aset tidak lancar": 81765000000000.0,
            "Jumlah liabilitas jangka pendek": 14300000000000.0,
            "Jumlah liabilitas jangka panjang": 1989000000000.0,
            "Jumlah ekuitas": 84714000000000.0,
            "Jumlah liabilitas dan ekuitas": 101003000000000.0,
            "Pendapatan bersih": 108249000000000.0,
            "Beban pokok pendapatan": -97738000000000.0,
            "Laba bruto": 10511000000000.0,
            "Laba sebelum pajak penghasilan": 22136000000000.0,
            "Beban pajak penghasilan": -475000000000.0,
            "Laba tahun berjalan": 21661000000000.0,
            "Jumlah aset": 19238000000000.0,
            "Piutang usaha": 1000000000.0,
            "Aset tetap": 156970

## Catatan Akhir: Efisiensi, Keterbatasan, dan Pengembangan Lanjutan

Notebook ini menyediakan alur kerja yang komprehensif untuk parsing dokumen dan ekstraksi informasi keuangan dasar. Namun, ada beberapa hal yang perlu diperhatikan:

*   **Efisiensi Pemrosesan**:
    *   **PDF Besar**: Seperti yang disebutkan, PDF besar dengan banyak halaman gambar bisa lambat karena OCR. Fitur **OCR Paralel** yang diimplementasikan di `parser_pdf.py` membantu mengurangi waktu tunggu.
    *   **Caching**: Mekanisme **caching** untuk `parser_pdf.py` (disimpan di `.cache_parser_dokumen` secara default) akan sangat membantu jika Anda sering memproses ulang dokumen yang sama, karena hasil parsing akan diambil dari cache jika file tidak berubah.
    *   **Pra-pemrosesan Gambar**: Langkah ini penting untuk akurasi OCR, tetapi juga menambah waktu pemrosesan untuk setiap gambar/halaman gambar.

*   **Akurasi Ekstraksi Kata Kunci dan Nilai**:
    *   **Logika Tahun Terbaru**: `pengekstrak_kata_kunci.py` kini mencoba mengidentifikasi tahun pelaporan dan memprioritaskan nilai yang berasosiasi dengan tahun tersebut, serta membedakannya dari nilai tahun sebelumnya. Akurasi logika ini sangat bergantung pada konsistensi format tabel dan layout dalam dokumen. Mungkin memerlukan penyesuaian regex lebih lanjut untuk berbagai format laporan keuangan.
    *   **Variasi Kata Kunci**: Keberhasilan ekstraksi juga bergantung pada seberapa komprehensif daftar `variasi` untuk setiap `kata_dasar` dalam `konfigurasi_kata_kunci_target`.
    *   **Normalisasi Nilai**: Fungsi `normalisasi_nilai_keuangan` menangani format umum Indonesia, tetapi format yang sangat tidak standar mungkin memerlukan penyesuaian.
    *   **Konteks**: Ekstraktor saat ini menggunakan konteks kalimat dan kedekatan dengan tahun. Untuk kasus yang sangat ambigu, pemahaman struktur tabel atau elemen visual mungkin diperlukan (di luar cakupan saat ini).

*   **Keterbatasan Bahasa Indonesia di NLTK**:
    *   **Stopwords**: `pengekstrak_kata_kunci.py` mencoba menggunakan stopwords Bahasa Indonesia dari NLTK. Pastikan resource ini terinstal (`nltk.download('stopwords')`).
    *   **Lemmatization/Stemming**: `WordNetLemmatizer` NLTK tidak dioptimalkan untuk Bahasa Indonesia. Untuk hasil yang lebih baik dalam normalisasi kata, pertimbangkan untuk mengintegrasikan stemmer khusus Bahasa Indonesia seperti PySastrawi (memerlukan instalasi terpisah). Saat ini, keakuratan pencocokan lebih bergantung pada variasi eksplisit yang disediakan.

*   **Pengembangan Lanjutan yang Mungkin Dilakukan**:
    *   Integrasi stemmer Bahasa Indonesia.
    *   Pengembangan logika yang lebih canggih untuk memahami struktur tabel dalam dokumen.
    *   Pelatihan model Machine Learning kustom untuk klasifikasi teks atau Named Entity Recognition (NER) pada dokumen keuangan untuk identifikasi entitas dan nilai yang lebih robust.
    *   Antarmuka pengguna grafis (GUI) atau aplikasi web di atas logika ini.
